In [2]:
from libcom import get_composite_image
import os
import numpy as np
import json
from PIL import Image
import cv2

def reconstruct(occ_level, image_path, predictor, save_dir):
    
    # get annotation
    img_id = image_path.split('/')[-1].split('.')[0]
    
    # get occluder bbox from annotation
    anno_path = f'realistic/annotations/{occ_level}/{img_id}.json'
    comp_path = f'realistic/images/{occ_level}/{img_id}.jpg'
    
    bg_path = f'BG/{img_id}.JPEG'
    
    with open(anno_path) as f:
        d = json.load(f)
        
    bbox = d['occluder_box']
    
    # get the center of the occluder bbox
    center_x = (bbox[0] + bbox[2]) // 2  # Average of x1 and x2
    center_y = (bbox[1] + bbox[3]) // 2  # Average of y1 and y2
    center_point = np.array([[center_x, center_y]])
    
    input_label = np.array([1])
    
    predictor.set_image(np.array(Image.open(comp_path)))
    masks, scores, logits = predictor.predict(
        point_coords=center_point,
        box=np.array([bbox]),
        point_labels=input_label,
        multimask_output=False
    )
    
    mask_image = (masks[0].astype(np.uint8) * 255)

    # Save the mask as a PNG
    Image.fromarray(mask_image).save('temp_mask.png')
    
    comp, _ = get_composite_image(comp_path, 'temp_mask.png',  bg_path, bbox, 'gaussian')
    
    save_path = os.path.join(save_dir, 'images', occ_level, f'{img_id}.jpg')
    
    cv2.imwrite(save_path, comp)

In [3]:
import os

save_dir = './reconstructed'

occ_ranges = ['low', 'medium', 'high']

for range_label in occ_ranges:
    os.makedirs('./reconstructed/images/' + range_label, exist_ok=True)

In [4]:

device = "cuda"
from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["vit_h"](checkpoint="./sam_vit_h_4b8939.pth")
sam.to(device)
predictor = SamPredictor(sam)

with open('file_list.txt', 'r') as names:
    for name in names:
        occ_level, bg_img_path = name.strip().split(' ')
        reconstruct(occ_level, bg_img_path, predictor, save_dir)